In [20]:
import os

dataset_path = r"C:\Users\Noor\Desktop\project_folder\Dataset"
subfolders = os.listdir(dataset_path)

print("Subfolders:")
for folder in subfolders:
    print(folder)


Subfolders:
Test
Train
Val


In [21]:
pip install torch torchvision

Note: you may need to restart the kernel to use updated packages.


In [22]:
train_dir = os.path.join(dataset_path, "Train")
valid_dir = os.path.join(dataset_path, "Val")
test_dir  = os.path.join(dataset_path, "Test")


In [23]:
import os
import torch
from torch import nn, optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader



# ✅ Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [24]:
import os
import torch
from torch import nn, optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader

# ✅ Transforms
transform = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ]),
}


train_dataset = datasets.ImageFolder(train_dir, transform=transform['train'])
valid_dataset = datasets.ImageFolder(valid_dir, transform=transform['valid'])
test_dataset  = datasets.ImageFolder(test_dir,  transform=transform['test'])

# ✅ DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)
test_loader  = DataLoader(test_dataset,  batch_size=32, shuffle=False)

# ✅ Load pretrained ResNet-50
model = models.resnet50(pretrained=True)

# ✅ Freeze early layers (optional)
for param in model.parameters():
    param.requires_grad = False

# ✅ Replace final layer
num_classes = len(train_dataset.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)

model = model.to(device)

# ✅ Loss & Optimizer
criterion = nn.CrossEntropyLoss()    
epochs =25
for epoch in range(epochs):
    model.train()
    running_loss = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs} - Loss: {running_loss/len(train_loader):.4f}")

# ✅ Save the model
torch.save(model.state_dict(), "resnet50_finetuned.pth")


KeyboardInterrupt: 

In [ ]:
import torch
from torchvision import models
from tqdm import tqdm  # progress bar

# ✅ Load the model
model = models.resnet50(pretrained=False)
num_classes = len(train_dataset.classes)
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)

model.load_state_dict(torch.load("resnet50_finetuned.pth", map_location=device))
model.to(device)
model.eval()

# ✅ Class names
class_names = train_dataset.classes

# ✅ Inference on test dataset
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in tqdm(test_loader, desc="Predicting"):
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# ✅ Show accuracy
correct = sum([p == t for p, t in zip(all_preds, all_labels)])
total = len(all_labels)
accuracy = correct / total * 100
print(f"\nTest Accuracy: {accuracy:.2f}%")

# ✅ Optional: Show predictions with class names
for i in range(len(all_preds)):
    true_label = class_names[all_labels[i]]
    pred_label = class_names[all_preds[i]]
    print(f"Image {i+1}: True = {true_label} | Predicted = {pred_label}")


In [26]:
!conda install pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia


^C
